In [1]:
import numpy as np

from ase import Atoms
from ase.io import read, write

from pyace import create_multispecies_basis_config
from pyace.activelearning import compute_B_projections

In [2]:
basis_config = {
  "deltaSplineBins": 0.001,
  "elements": ['Au'],

  "embeddings": {
    "ALL": {
      "npot": 'FinnisSinclairShiftedScaled', # ?
      "fs_parameters": [ 1, 1],
      "ndensity": 1, # ?
    },
  },

  "bonds": {
    "ALL": {
      "radbase": "SBessel",
      "radparameters": [ 5.25 ],
      "rcut": 6, # ?
      "dcut": 0.01, # ?
    }
  },

  "functions": {
    # "number_of_functions_per_element": 1000,
    "ALL": {
        "nradmax_by_orders": [ 8,8,6],
        "lmax_by_orders"   : [ 0,6,4] }
  }
}

# basis_config = {
#   "deltaSplineBins": 0.001,
#   "elements": ["Au"],

#   "embeddings": {
#     "ALL": {
#       "npot": "FinnisSinclairShiftedScaled",
#       "fs_parameters": [1, 1],   # length must be 2*ndensity
#       "ndensity": 1
#     }
#   },

#   "bonds": {
#     "ALL": {
#       "radbase": "SBessel",
#       "radparameters": [5.0],
#       "rcut": 5.0,     # ≥ interlayer 3.35 Å and >> C–C 1.42 Å; near GAP-20 SOAP ~4.5 Å
#       "dcut": 0.05
#     }
#   },

#   "functions": {
#     "ALL": {
#       "nradmax_by_orders": [12, 10, 8],  # 2-, 3-, (effective) 4-body
#       "lmax_by_orders"   : [ 0,  8, 6]   # scalar 2-body; richer angular detail for 3/4-body
#     }
#   }
# }

In [2]:

basis_config = {
    "deltaSplineBins": 0.005,
    "elements": ['Au'],

    "embeddings": {
        "ALL": {
        "npot": 'FinnisSinclairShiftedScaled',
        "fs_parameters": [1.5860537, 2.0330848],
        "ndensity": 2.1012853,
        },
    },

    "bonds": {
        "ALL": {
        "radbase": "SBessel",
        "radparameters": [4.8485478],
        "rcut": 4.9536137,
        "dcut": 0.005,
        }
    },

    "functions": {
        # "number_of_functions_per_element": 1000,
        "ALL": {
            "nradmax_by_orders": [4.0, 4.0926433, 2.0],
            "lmax_by_orders"   : [0.4406132, 3.2915619, 8.0]
        }
    }
}

from copy import deepcopy

def params_to_exact_config(basis_config,
                           ndensity_bounds=(1, 3),
                           nrad_bounds=((4, 12), (4, 12), (2, 10)),
                           lmax_bounds=((0, 2), (2, 10), (2, 8))):
    """
    Snap an existing basis_config to the same integer/clipping/order constraints
    used in make_basis_config, reproducing the exact config that would have been
    used during tuning.

    - Rounds/clips ndensity, nradmax_by_orders, lmax_by_orders
    - Enforces descending sort for nrad list
    - Enforces l2 >= l1 and l3 <= l2 for lmax list
    - Rebuilds fs_parameters to length 2*ndensity from the first two entries
    - Leaves floats like rcut/dcut/radparameters/deltaSplineBins untouched
    """
    def iround(x, lo, hi):
        return int(max(lo, min(hi, round(float(x)))))

    cfg = deepcopy(basis_config)

    # --- embeddings ---
    emb_all = cfg.setdefault("embeddings", {}).setdefault("ALL", {})

    # ndensity: round/clip to bounds (default to infer from fs_parameters if present)
    nd_lo, nd_hi = ndensity_bounds
    ndensity_val = emb_all.get("ndensity", None)
    if ndensity_val is None:
        # infer from fs_parameters length if possible
        fs = emb_all.get("fs_parameters", [])
        ndensity_val = max(nd_lo, min(nd_hi, len(fs) // 2)) if len(fs) >= 2 else nd_lo
    ndensity = iround(ndensity_val, nd_lo, nd_hi)
    emb_all["ndensity"] = ndensity  # ensure int

    # fs_parameters: rebuild to exactly length 2*ndensity from first two numbers
    fs_src = emb_all.get("fs_parameters", [])
    if len(fs_src) == 0:
        base_p0, base_p1 = 1.0, 1.0
    elif len(fs_src) == 1:
        base_p0, base_p1 = float(fs_src[0]), float(fs_src[0])
    else:
        base_p0, base_p1 = float(fs_src[0]), float(fs_src[1])

    fs_parameters = []
    for _ in range(ndensity):
        fs_parameters.extend([base_p0, base_p1])
    emb_all["fs_parameters"] = fs_parameters  # exact length: 2*ndensity

    # npot/radbase: keep whatever strings are present
    # (no change needed unless you want to clamp to allowed choices)

    # --- bonds ---
    # Keep floats as-is; just ensure they’re floats if present
    bonds_all = cfg.setdefault("bonds", {}).setdefault("ALL", {})
    if "rcut" in bonds_all:
        bonds_all["rcut"] = float(bonds_all["rcut"])
    if "dcut" in bonds_all:
        bonds_all["dcut"] = float(bonds_all["dcut"])
    if "radparameters" in bonds_all and bonds_all["radparameters"]:
        bonds_all["radparameters"] = [float(bonds_all["radparameters"][0])]

    # --- functions ---
    fun_all = cfg.setdefault("functions", {}).setdefault("ALL", {})

    # nradmax_by_orders: round/clip each, then sort descending
    nr_lohi = nrad_bounds
    nrad = fun_all.get("nradmax_by_orders", [8, 8, 6])
    # pad/trim to 3
    nrad = (list(nrad) + [nrad[-1]] * 3)[:3] if nrad else [8, 8, 6]
    n1 = iround(nrad[0], *nr_lohi[0])
    n2 = iround(nrad[1], *nr_lohi[1])
    n3 = iround(nrad[2], *nr_lohi[2])
    n1, n2, n3 = sorted([n1, n2, n3], reverse=True)
    fun_all["nradmax_by_orders"] = [n1, n2, n3]

    # lmax_by_orders: round/clip each, then enforce l2>=l1 and l3<=l2
    lm_lohi = lmax_bounds
    lmax = fun_all.get("lmax_by_orders", [0, 6, 4])
    lmax = (list(lmax) + [lmax[-1]] * 3)[:3] if lmax else [0, 6, 4]
    l1 = iround(lmax[0], *lm_lohi[0])
    l2 = iround(lmax[1], *lm_lohi[1]); l2 = max(l2, l1)
    l3 = iround(lmax[2], *lm_lohi[2]); l3 = min(l3, l2)
    fun_all["lmax_by_orders"] = [l1, l2, l3]

    # --- misc ---
    if "deltaSplineBins" in cfg:
        cfg["deltaSplineBins"] = float(cfg["deltaSplineBins"])
    if "elements" in cfg:
        cfg["elements"] = list(cfg["elements"])  # ensure list

    return cfg


basis_config = params_to_exact_config(basis_config)

In [3]:
basis = create_multispecies_basis_config(basis_config)
basis

BBasisConfiguration(deltaSplineBins=0.001, funcspecs_blocks=['Au', ])

### Compare FCC structures with 1% strain and unstrained

In [4]:
from ase.build import bulk, make_supercell
import numpy as np

a1 = 3.58
supercell = 1
fcc1 = bulk('Au', 'fcc', a=a1, cubic=True)

fcc2 = fcc1.copy()
fcc2.set_cell(0.99 * fcc2.cell, scale_atoms=True)

# fcc1 = make_supercell(fcc1, np.eye(3) * supercell)
# fcc2 = make_supercell(fcc2, np.eye(3) * supercell)

x1 = compute_B_projections(basis, [fcc1])[0]
x2 = compute_B_projections(basis, [fcc2])[0]

distance = np.linalg.norm(x1[0] - x2[0])
print(distance)

47.725757408845844
